In [2]:
from datetime import datetime
from typing import Optional

import sqlalchemy as sa
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped, mapped_column
from sqlalchemy.ext.asyncio import AsyncAttrs

from fazbot.db import BaseDatabase


class Base(DeclarativeBase): ...


In [ ]:
db = BaseDatabase(
    'mariadb+asyncmy',
    'test',
    'test',
    'localhost',
    'test_fazbot'
)

In [4]:
class WhitelistedGuild(Base, AsyncAttrs):
    __tablename__ = "whitelisted_guild"

    guild_id: Mapped[int] = mapped_column(primary_key=True)
    guild_name: Mapped[str] = mapped_column(sa.String(32))
    from_: Mapped[datetime] = mapped_column(name="from")
    until: Mapped[Optional[datetime]] = mapped_column(default=None)

In [ ]:
WhitelistedGuild.guild_id.primary_key

In [5]:
print(sa.schema.CreateTable(WhitelistedGuild.__table__))


CREATE TABLE whitelisted_guild (
	guild_id INTEGER NOT NULL, 
	guild_name VARCHAR(32) NOT NULL, 
	"from" DATETIME NOT NULL, 
	until DATETIME, 
	PRIMARY KEY (guild_id)
)




In [ ]:
pwr1 = WhitelistedGuild(
    guild_id=921720674823839804,
    guild_name="ao",
    from_=datetime.now(),
    until=None,
)

pwr2 = WhitelistedGuild(
    guild_id=921720674823839805,
    guild_name="ab",
    from_=datetime.now(),
    until=None,
)


async with db.enter_session() as session:
    session.add_all([pwr1, pwr2])
    await session.commit()

In [ ]:
async with db.enter_connection() as conn:
    cursor = await conn.execute(sa.text("SELECT * FROM whitelisted_guild"))
    results = cursor.all()
    print(results)

[(921720674823839804, 'ao', datetime.datetime(2024, 6, 15, 23, 39, 40), None), (921720674823839805, 'ab', datetime.datetime(2024, 6, 15, 23, 39, 40), None)]


In [ ]:
async with db.enter_connection() as conn:
    cursor = await conn.execute(sa.text("TRUNCATE whitelisted_guild"))